In [1]:
%%capture
import os
import site
% os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
%cd /home/schirrmr/


In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('DEBUG')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)

In [3]:
from braindecode.torch_ext.util import set_random_seeds, np_to_var, var_to_np
from braindecode.models.hybrid import HybridNet
from braindecode.models.util import to_dense_prediction_model
from torch import nn

set_random_seeds(seed=0, cuda=True)

n_classes = 5
n_chans = 64
input_time_length = 1000
model = HybridNet(n_chans, n_classes, input_time_length)



2018-07-24 10:13:22,433 DEBUG : $HOME=/home/lukas
2018-07-24 10:13:22,433 DEBUG : matplotlib data path /home/lukas/anaconda3/envs/braindecode/lib/python3.6/site-packages/matplotlib/mpl-data
2018-07-24 10:13:22,435 DEBUG : loaded rc file /home/lukas/anaconda3/envs/braindecode/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc
2018-07-24 10:13:22,436 DEBUG : matplotlib version 2.2.2
2018-07-24 10:13:22,437 DEBUG : interactive is False
2018-07-24 10:13:22,437 DEBUG : platform is linux
2018-07-24 10:13:22,438 DEBUG : loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'site', 'os', 'errno', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig',

/home/lukas/PycharmProjects/braindecode/braindecode/models/deep4.py:150: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(model.conv_time.weight, gain=1)
/home/lukas/PycharmProjects/braindecode/braindecode/models/deep4.py:153: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(model.conv_time.bias, 0)
/home/lukas/PycharmProjects/braindecode/braindecode/models/deep4.py:155: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(model.conv_spat.weight, gain=1)
/home/lukas/PycharmProjects/braindecode/braindecode/models/deep4.py:159: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(model.bnorm.weight, 1)
/home/lukas/PycharmProjects/braindecode/braindecode/models/deep4.py:160: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(model.bnorm.

### Get braindevel parameter values and braindecode parameters

In [4]:
params = np.load(
    './1.2017-05-31_11-14-00.model_params.npy',
    encoding='bytes',)

network = model.create_network()
state_dict = network.state_dict()

### Reassign parameter values

In [17]:
# params[3] # ist nicht mehr existierender bias, löschen und von bnorm running mean (params[6]) abziehen
# lasange logic: beta(bias), gamma(weight), running mean, running var

# state_dict['reduced_deep_model.conv_time.weight'] = np_to_var(params[0], dtype=np.float32)
# state_dict['reduced_deep_model.conv_time.bias'] = np_to_var(params[1], dtype=np.float32)
# state_dict['reduced_deep_model.conv_spat.weight'] = np_to_var(params[2], dtype=np.float32)
# state_dict['reduced_deep_model.bnorm.weight'] = np_to_var(params[5], dtype=np.float32)
# state_dict['reduced_deep_model.bnorm.bias'] = np_to_var(params[4], dtype=np.float32)
# state_dict['reduced_deep_model.bnorm.running_mean'] = np_to_var(params[6]-params[3], dtype=np.float32)
# state_dict['reduced_deep_model.bnorm.running_var'] = np_to_var(params[7], dtype=np.float32)
# 
# state_dict['reduced_deep_model.conv_2.weight'] = np_to_var(params[8], dtype=np.float32)
# state_dict['reduced_deep_model.bnorm_2.weight'] = np_to_var(params[11], dtype=np.float32)
# state_dict['reduced_deep_model.bnorm_2.bias'] = np_to_var(params[10], dtype=np.float32)
# state_dict['reduced_deep_model.bnorm_2.running_mean'] = np_to_var(params[12]-params[9], dtype=np.float32)
# state_dict['reduced_deep_model.bnorm_2.running_var'] = np_to_var(params[13], dtype=np.float32)
#     
# state_dict['reduced_deep_model.conv_3.weight'] = np_to_var(params[14], dtype=np.float32)
# state_dict['reduced_deep_model.bnorm_3.weight'] = np_to_var(params[17], dtype=np.float32)
# state_dict['reduced_deep_model.bnorm_3.bias'] = np_to_var(params[16], dtype=np.float32)
# state_dict['reduced_deep_model.bnorm_3.running_mean'] = np_to_var(params[18]-params[15], dtype=np.float32)
# state_dict['reduced_deep_model.bnorm_3.running_var'] = np_to_var(params[19], dtype=np.float32)
# 
# state_dict['reduced_deep_model.conv_4.weight'] = np_to_var(params[20], dtype=np.float32)
# state_dict['reduced_deep_model.bnorm_4.weight'] = np_to_var(params[23], dtype=np.float32)
# state_dict['reduced_deep_model.bnorm_4.bias'] = np_to_var(params[22], dtype=np.float32)
# state_dict['reduced_deep_model.bnorm_4.running_mean'] = np_to_var(params[24]-params[21], dtype=np.float32)
# state_dict['reduced_deep_model.bnorm_4.running_var'] = np_to_var(params[25], dtype=np.float32)
#     
# state_dict['reduced_deep_model.deep_final_conv.weight'] = np_to_var(params[26], dtype=np.float32)
# state_dict['reduced_deep_model.deep_final_conv.bias'] = np_to_var(params[27], dtype=np.float32)
#    
# state_dict['reduced_shallow_model.conv_time.weight'] = np_to_var(params[28], dtype=np.float32)
# state_dict['reduced_shallow_model.conv_time.bias'] = np_to_var(params[29], dtype=np.float32)
# state_dict['reduced_shallow_model.conv_spat.weight'] = np_to_var(params[30], dtype=np.float32)
# state_dict['reduced_shallow_model.bnorm.weight'] = np_to_var(params[33], dtype=np.float32)
# state_dict['reduced_shallow_model.bnorm.bias'] = np_to_var(params[32], dtype=np.float32)
# state_dict['reduced_shallow_model.bnorm.running_mean'] = np_to_var(params[34]-params[31], dtype=np.float32)
# state_dict['reduced_shallow_model.bnorm.running_var'] = np_to_var(params[35], dtype=np.float32)
# 
# state_dict['reduced_shallow_model.shallow_final_conv.weight'] = np_to_var(params[36], dtype=np.float32)
# state_dict['reduced_shallow_model.shallow_final_conv.bias'] = np_to_var(params[37], dtype=np.float32)
# 
# state_dict['final_conv.weight'] = np_to_var(params[38].T[:,:,None,None], dtype=np.float32)
# state_dict['final_conv.bias'] = np_to_var(params[39], dtype=np.float32)


# logic from lukas' code: running mean, running var, gamma(weight), beta(bias)
state_dict['reduced_deep_model.conv_time.weight'] = np_to_var(params[0], dtype=np.float32)
state_dict['reduced_deep_model.conv_time.bias'] = np_to_var(params[1], dtype=np.float32)
state_dict['reduced_deep_model.conv_spat.weight'] = np_to_var(params[2], dtype=np.float32)
state_dict['reduced_deep_model.bnorm.weight'] = np_to_var(params[6], dtype=np.float32)
state_dict['reduced_deep_model.bnorm.bias'] = np_to_var(params[7], dtype=np.float32)
state_dict['reduced_deep_model.bnorm.running_mean'] = np_to_var(params[4]-params[3], dtype=np.float32)
state_dict['reduced_deep_model.bnorm.running_var'] = np_to_var(params[5], dtype=np.float32)

state_dict['reduced_deep_model.conv_2.weight'] = np_to_var(params[8], dtype=np.float32)
state_dict['reduced_deep_model.bnorm_2.weight'] = np_to_var(params[12], dtype=np.float32)
state_dict['reduced_deep_model.bnorm_2.bias'] = np_to_var(params[13], dtype=np.float32)
state_dict['reduced_deep_model.bnorm_2.running_mean'] = np_to_var(params[10]-params[9], dtype=np.float32)
state_dict['reduced_deep_model.bnorm_2.running_var'] = np_to_var(params[11], dtype=np.float32)
    
state_dict['reduced_deep_model.conv_3.weight'] = np_to_var(params[14], dtype=np.float32)
state_dict['reduced_deep_model.bnorm_3.weight'] = np_to_var(params[18], dtype=np.float32)
state_dict['reduced_deep_model.bnorm_3.bias'] = np_to_var(params[19], dtype=np.float32)
state_dict['reduced_deep_model.bnorm_3.running_mean'] = np_to_var(params[16]-params[15], dtype=np.float32)
state_dict['reduced_deep_model.bnorm_3.running_var'] = np_to_var(params[17], dtype=np.float32)

state_dict['reduced_deep_model.conv_4.weight'] = np_to_var(params[20], dtype=np.float32)
state_dict['reduced_deep_model.bnorm_4.weight'] = np_to_var(params[24], dtype=np.float32)
state_dict['reduced_deep_model.bnorm_4.bias'] = np_to_var(params[25], dtype=np.float32)
state_dict['reduced_deep_model.bnorm_4.running_mean'] = np_to_var(params[22]-params[21], dtype=np.float32)
state_dict['reduced_deep_model.bnorm_4.running_var'] = np_to_var(params[23], dtype=np.float32)
    
state_dict['reduced_deep_model.deep_final_conv.weight'] = np_to_var(params[26], dtype=np.float32)
state_dict['reduced_deep_model.deep_final_conv.bias'] = np_to_var(params[27], dtype=np.float32)
   
state_dict['reduced_shallow_model.conv_time.weight'] = np_to_var(params[28], dtype=np.float32)
state_dict['reduced_shallow_model.conv_time.bias'] = np_to_var(params[29], dtype=np.float32)
state_dict['reduced_shallow_model.conv_spat.weight'] = np_to_var(params[30], dtype=np.float32)
state_dict['reduced_shallow_model.bnorm.weight'] = np_to_var(params[34], dtype=np.float32)
state_dict['reduced_shallow_model.bnorm.bias'] = np_to_var(params[35], dtype=np.float32)
state_dict['reduced_shallow_model.bnorm.running_mean'] = np_to_var(params[32]-params[31], dtype=np.float32)
state_dict['reduced_shallow_model.bnorm.running_var'] = np_to_var(params[33], dtype=np.float32)

state_dict['reduced_shallow_model.shallow_final_conv.weight'] = np_to_var(params[36], dtype=np.float32)
state_dict['reduced_shallow_model.shallow_final_conv.bias'] = np_to_var(params[37], dtype=np.float32)

state_dict['final_conv.weight'] = np_to_var(params[38].T[:,:,None,None], dtype=np.float32)
state_dict['final_conv.bias'] = np_to_var(params[39], dtype=np.float32)

### Verify that we have not made any gross errors during the assignment

In [18]:
np.all([(value_mod.size() == value_unmod.size()) for (_, value_mod), (_, value_unmod) in zip(state_dict.items(), network.state_dict().items())])

True

### Assign the new parameter values to the model

In [19]:
network.load_state_dict(state_dict, strict=True)

### Load data recorded directly after parameters were trained to test if decoding is above chance

In [27]:
data = np.load('./2017-05-31_11-29-40.npy')

### Decode

In [28]:
import torch as th
th.cuda.device(1)

In [29]:
data = data[2000:3000,:]
labels = data[:,-1]
data = np_to_var(data[None,:,:-1,None]).cuda()
network.cuda()

HybridNet(
  (reduced_deep_model): Sequential(
    (dimshuffle): Expression(expression=_transpose_time_to_spat)
    (conv_time): Conv2d(1, 20, kernel_size=(10, 1), stride=(1, 1))
    (conv_spat): Conv2d(20, 30, kernel_size=(1, 64), stride=(1, 1), bias=False)
    (bnorm): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_nonlin): Expression(expression=elu)
    (pool): MaxPool2d(kernel_size=(3, 1), stride=(1, 1), padding=0, dilation=(1, 1), ceil_mode=False)
    (pool_nonlin): Expression(expression=identity)
    (drop_2): Dropout(p=0.5)
    (conv_2): Conv2d(30, 40, kernel_size=(10, 1), stride=(1, 1), dilation=(3, 1), bias=False)
    (bnorm_2): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (nonlin_2): Expression(expression=elu)
    (pool_2): MaxPool2d(kernel_size=(3, 1), stride=(1, 1), padding=0, dilation=(3, 1), ceil_mode=False)
    (pool_nonlin_2): Expression(expression=identity)
    (drop_3): Dropout(p=0.5)
  

In [30]:
data = data.permute(0,2,1,3)

In [31]:
outputs = model(data)

In [32]:
predicted_labels = np.argmax(var_to_np(outputs), axis=1)
accuracy = np.mean(labels  == predicted_labels)
print("Accuracy: {:.1f}%".format(
    accuracy * 100))

Accuracy: 0.0%


/home/lukas/anaconda3/envs/braindecode/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  


In [33]:
labels

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [34]:
predicted_labels

array([[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 

In [35]:
outputs[0,:,0]

tensor([ -6.8772,  -3.8962,  -6.2920, -12.1759,  -0.0235], device='cuda:0')